<a href="https://colab.research.google.com/github/mortezaaghajanzadeh/Machine-learning-in-Finance/blob/main/Lecture%201/introduction_to_tensorflow_lecture_1_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture 1 (Part 1): Introduction to TensorFlow in Python.**
### Based on code from Chapter 1 in ``Machine Learning for Economics and Finance in TensorFlow 2'' (Hull, 2021).

In [ ]:
# Import libraries.
import tensorflow as tf
import numpy as np
import pandas as pd

## **Listing 1-2.** Implement OLS.

In [ ]:
print(tf.__version__)

# Define the data as constants.
X = tf.constant([[1, 0], [1, 2]], tf.float32)
Y = tf.constant([[2], [4]], tf.float32)

# Matrix multiply X by X’s transpose and invert.
beta_0 = tf.linalg.inv(tf.matmul(tf.transpose(X), X))

# Matrix multiply beta_0 by X’s transpose.
beta_1 = tf.matmul(beta_0, tf.transpose(X))

# Matrix multiply beta_1 by Y.
beta = tf.matmul(beta_1, Y)

2.14.0
[[2.]
 [1.]]


## **Listing 1-4.** Print tensors.

In [ ]:
# Print the feature matrix.
print(X)

# Print the coefficient vector.
print(beta.numpy())

tf.Tensor(
[[1. 0.]
 [1. 2.]], shape=(2, 2), dtype=float32)
[[2.]
 [1.]]


## **Listing 1-6.** Generate OLS predictions with static graphs.

In [ ]:
# Define OLS prediction function as static graph.
@tf.function
def ols_predict(X, beta):
	y_hat = tf.matmul(X, beta)
	return y_hat

# Predict Y using X and beta.
predictions = ols_predict(X, beta)

# Print predictions.
print(predictions)

tf.Tensor(
[[2.]
 [4.]], shape=(2, 1), dtype=float32)


## **Listing 1-7.** Solve an OLS model with tf.keras().

In [ ]:
# Define sequential model.
ols = tf.keras.Sequential()

# Add dense layer with linear activation.
ols.add(tf.keras.layers.Dense(1, input_shape = (2,),
    use_bias = False , activation = 'linear'))

# Set optimizer and loss.
ols.compile(optimizer = 'SGD', loss = 'mse')

# Train model.
ols.fit(X, Y, epochs = 500)

# Print parameter estimates.
print(ols.weights[0].numpy())

Epoch 1/10
1/1 [==============================] - 1s 822ms/step - loss: 8.4241
Epoch 2/10
1/1 [==============================] - 0s 21ms/step - loss: 7.5676
Epoch 3/10
1/1 [==============================] - 0s 25ms/step - loss: 6.7984
Epoch 4/10
1/1 [==============================] - 0s 24ms/step - loss: 6.1077
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 5.4873
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 4.9302
Epoch 7/10
1/1 [==============================] - 0s 26ms/step - loss: 4.4298
Epoch 8/10
1/1 [==============================] - 0s 10ms/step - loss: 3.9804
Epoch 9/10
1/1 [==============================] - 0s 28ms/step - loss: 3.5769
Epoch 10/10
1/1 [==============================] - 0s 21ms/step - loss: 3.2144
[[1.2289267 ]
 [0.24720308]]


## **Listing 1-8.** Solve an OLS model with tf.estimator().

In [ ]:
# Define feature columns.
features = [
tf.feature_column.numeric_column("constant"),
tf.feature_column.numeric_column("x1")
]

# Define model.
ols = tf.estimator.LinearRegressor(features)

# Define function to feed data to model.
def train_input_fn():
	features = {"constant": [1, 1], "x1": [0, 2]}
	target = [2, 4]
	return features, target

# Train OLS model.
ols.train(train_input_fn, steps = 100)

## **Listing 1-9.** Make predictions with an OLS model with tf.estimator().

In [ ]:
# Define feature columns.
def test_input_fn():
    features = {"constant": [1, 1], "x1": [3, 5]}
    return features

# Define prediction generator.
predict_gen = ols.predict(input_fn=test_input_fn)

# Generate predictions.
predictions = [next(predict_gen) for j in range(2)]

# Print predictions.
print(predictions)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


[{'predictions': array([5.0000067], dtype=float32)}, {'predictions': array([7.000059], dtype=float32)}]


## **Listing 1-10.** List all available devices, select CPU, and then switch to GPU.

In [ ]:
# Print list of devices.
devices = tf.config.list_physical_devices()
print(devices)

# Set device to CPU.
tf.config.experimental.set_visible_devices(
devices[0], 'CPU')

# Change device to GPU.
tf.config.experimental.set_visible_devices(
devices[3], 'GPU')